# Build a Crew to Tailor Job Applications

In [1]:
# Warning control
import os
import warnings
from dotenv import load_dotenv,find_dotenv

warnings.filterwarnings('ignore')
load_dotenv(find_dotenv())

True

In [2]:
from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
  project_name="my-crewai-vertex-app", # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: my-crewai-vertex-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
# Run gcloud auth application-default login before running this script
# to set up the credentials for the Google Cloud SDK
# and the Google Cloud client libraries.
# SERPER API key is required for this example. 
# You can set it up in your environment variables or directly in the code.


from crewai import LLM
from google.auth import default

credentials, _ = default()

llm = LLM(
    model="vertex_ai/gemini-2.5-flash-preview-05-20",
    temperature=0.7,
    # credentials=credentials,
    vertex_project=os.getenv("GOOGLE_CLOUD_PROJECT_ID"),
    vertex_location=os.getenv("GOOGLE_CLOUD_PROJECT_REGION"),
)

llm.call("Hello there Gemini! How are you doing today?")


os.environ["OPENAI_API_KEY"] = os.environ["GROQ_API_KEY"] 
os.environ["OPENAI_API_BASE"] = os.environ["GROQ_API_BASE"] 
os.environ["OPENAI_MODEL_NAME"] = 'meta-llama/llama-4-scout-17b-16e-instruct'

- Import libraries, APIs and LLM

In [4]:
from crewai import Agent, Task, Crew

## crewAI Tools

In [5]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

/home/padmanabhan/Documents/Github/Agents-Lab/.venv/lib/python3.12/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [6]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

# Padmanabhan  
- Email: padmanabhan@example.com  
- Phone: +91 98765 43210  

## Profile  
Padmanabhan is a dynamic AI & Data Science Manager with over a decade of experience delivering business-impacting solutions across data engineering, machine learning, and analytics. With a Master’s degree in Computer Science (Big Data and Data Science specialization) from Simon Fraser University, Canada, he brings a unique blend of academic rigor and hands-on industry expertise.

He currently leads a cross-functional team of data professionals and a UI designer at Ford Motor Company, developing scalable AI solutions with a strong focus on Generative AI applications, automation, and intelligent dashboarding. His work spans predictive modeling, data pipelines, and decision-support systems aligned with business strategy.

Skilled in Python, R, SQL, and modern data tools such as Tableau, Power BI, TensorFlow, and PySpark, Padmanabhan is known for building robust end-to-end pipelines, synthesizing actionable insights, and leading high-impact data initiatives. He is proactive, people-oriented, and passionate about continuous learning and innovation in AI.

## Work History  

### Ford Motor Company — Chennai, India (Hybrid)  
**Technical Project Manager – AI/ML**  
*Oct 2022 – Present*  
- Leading end-to-end development of AI/ML applications and dashboards.  
- Focused on business-driven Generative AI use cases and automation solutions.  
- Managing a team of data scientists, engineers, and a UI/UX designer.  

**Lead Engineer – AI/ML**  
*Feb 2021 – Oct 2022*  
- Developed predictive models and time series analysis tools.  
- Built crash data analysis solutions and synthetic data generation models.  
- Created interactive dashboards to support engineering decisions.  

### Simon Fraser University — Burnaby, Canada  

**Big Data Analyst**  
*May 2020 – Sep 2020*  
- Delivered AI/ML solutions for clients via SFU Big Data Hub.  
- Developed object detection models in TensorFlow Lite.  
- Created client dashboards using Tableau and Python.  

**Big Data Analyst / Research Assistant**  
*Jan 2019 – May 2020*  
- Built data pipelines and deployed ML models using Sklearn and TensorFlow.  
- Developed data visualizations with Tableau and Plotly.  

**Graduate Teaching Assistant**  
*Jan 2019 – Apr 2020*  
- TA for courses like Big Data Programming and Database Systems.  
- Supported students with coursework and exam preparation.  

### Avcorp Industries — Vancouver, Canada  
**Data Analyst Intern (Co-op)**  
*May 2019 – Aug 2019*  
- Built data ingestion and cleaning pipelines using Python and SQL.  
- Designed ML models to detect equipment failures.  
- Partnered with engineers to enhance manufacturing data use.  

### Cognizant Technology Solutions — Chennai, India  
**Associate – DWBI & Analytics**  
*Sep 2012 – Jul 2018*  
- Worked with major clients like JPMorgan Chase & Co., American Express, and H-E-B.  
- Built ETL pipelines using Informatica, performed data modeling and integration.  
- Developed BI dashboards with QlikView and Tableau.  
- Won several internal awards for performance and innovation.  

## Education  

**Master of Science in Computer Science (Big Data & Data Science)**  
Simon Fraser University, Canada  

**Bachelor’s Degree (Engineering)**  
Anna University, Chennai, India

## Skills  
- Programming: Python, R, SQL, PySpark  
- ML & AI: Scikit-learn, TensorFlow, Generative AI, Time Series Modeling  
- Tools: Tableau, Power BI, QlikView, TensorFlow Lite, Informatica  
- Data Engineering: ETL, Data Pipelines, Data Modeling  
- Soft Skills: Project Management, Team Leadership, Stakeholder Communication  


## Creating Agents

In [7]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=llm
)

In [8]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm=llm
)

In [9]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=llm
)

In [10]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm=llm
)

## Creating Tasks

In [11]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
)

In [12]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [13]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [14]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [15]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [22]:
job_application_inputs = {
    'job_posting_url': 'https://aijobs.ai/job/senior-manager-responsible-ai-solutions-2',
    'github_url': 'https://github.com/padmanabhan-r',
    'personal_writeup': """Padmanabhan is an experienced AI & Data Science Manager with over 10 years of expertise in machine learning, data engineering, and business analytics. He holds a Master’s degree in Computer Science with a specialization in Big Data and Data Science from Simon Fraser University, Canada. Currently leading a cross-functional team at Ford Motor Company, he focuses on building scalable ML solutions, Generative AI applications, and intelligent data pipelines that drive business decisions. Padmanabhan has a proven track record of delivering end-to-end AI products, developing predictive models, and creating actionable insights through dashboards and automation. He is passionate about solving complex data problems and thrives in roles that require a blend of technical depth, strategic thinking, and leadership. With strong programming skills in Python, R, and SQL, and hands-on experience with tools like Tableau, Power BI, and TensorFlow, Padmanabhan is well-suited for senior AI/ML roles that demand innovation, ownership, and impact."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [23]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: bd406eb2-ee71-40e4-9c7b-16e0a13e906c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

I0000 00:00:1747926883.380048   19876 chttp2_transport.cc:1201] ipv6:%5B::1%5D:4317: Got goaway [11] err=UNAVAILABLE:GOAWAY received; Error code: 11; Debug Text: ping_timeout {grpc_status:14, http2_error:11, created_time:"2025-05-22T20:44:43.380034348+05:30"}


🚀 Crew: crew
└── 📋 Task: 35285f7d-7672-4542-b680-483d1b025534
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 35285f7d-7672-4542-b680-483d1b025534
       Status: Executing Task...
    └── 🤖 Agent: Tech Job Researcher
            Status: In Progress

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://aijobs.ai/job/senior-manager-responsible-ai-solutions-2) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.


...

# Agent: Engineering Interview Preparer
## Task: Create a set of potential interview questions and talking points based on the tailored resume and job requirements. Utilize tools to generate relevant questions and discussion points. Make sure to use these question and talking points to help the candiadte highlight the main points of the resume and how it matches the job posting.


# Agent: Engineering Interview Preparer
## Final Answer: 
The user wants me to act as an Engineering Interview Preparer. My goal is to create interview questions and talking points based on the provided resume (Padmanabhan) and job requirements (Senior Manager, Responsible AI Solutions - Ford Foundation).

I need to generate a document containing key questions and talking points that the candidate should prepare for the initial interview. The final answer must be the complete content, not a summary.

Here's my plan:
1.  **Analyze the Job Description:** Break down the key responsibilities, required technical s

🚀 Crew: crew
├── 📋 Task: 35285f7d-7672-4542-b680-483d1b025534
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: ✅ Completed
├── 📋 Task: 667f0579-5d19-4837-8a96-1cc3247c9096
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Personal Profiler for Engineers
│           Status: ✅ Completed
├── 📋 Task: e2805831-b7c4-403e-9a1c-3d27ebab6e50
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
└── 📋 Task: 428df95f-61e7-4198-94e8-d8353c26a210
       Status: Executing Task...
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 35285f7d-7672-4542-b680-483d1b025534
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: ✅ Completed
├── 📋 Task: 667f0579-5d19-4837-8a96-1cc3247c9096
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Personal Profiler for Engineers
│           Status: ✅ Completed
├── 📋 Task: e2805831-b7c4-403e-9a1c-3d27ebab6e50
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
└── 📋 Task: 428df95f-61e7-4198-94e8-d8353c26a210
       Assigned to: Engineering Interview Preparer
       Status: ✅ Completed
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 428df95f-61e7-4198-94e8-d8353c26a210                                                                     │
│  Agent: Engineering Interview Preparer                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: bd406eb2-ee71-40e4-9c7b-16e0a13e906c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Dislplay the generated `tailored_resume.md` file.

In [24]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

**Padmanabhan**
[City, State] | [Email Address] | [LinkedIn Profile URL] | [GitHub Profile URL]

**Summary**

Highly accomplished and strategic AI & Data Science Manager with over a decade of experience in machine learning, data engineering, and business analytics, currently excelling as an AI Engineer & Technical Project Manager at Ford Motor Company. Proven leader in developing and deploying scalable, cutting-edge Generative AI solutions and robust intelligent data pipelines that drive significant business impact and inform strategic decisions. Adept at managing complex AI projects from conception to production, defining KPIs, and optimizing model performance. Deeply committed to ethical AI practices, fairness, and transparency, seeking to leverage expertise to lead responsible AI initiatives at the Ford Foundation. Excels in translating complex technical information into clear business value for diverse stakeholders.

**Professional Experience**

**AI Engineer & Technical Project Manager** | Ford Motor Company | [Start Date] – Present
*   Led cross-functional teams in the end-to-end creation and deployment of scalable machine learning solutions, cutting-edge Generative AI applications, and intelligent data pipelines, directly influencing critical business decisions and driving operational efficiency.
*   Managed the full lifecycle of AI projects, including defining scope, allocating resources, managing timelines, and mitigating risks, ensuring on-time and within-budget delivery while maintaining high quality standards.
*   Spearheaded the development and implementation of sophisticated predictive models and Generative AI solutions, including those leveraging Retrieval-Augmented Generation (RAG) and exploring Agentic architectures, to solve complex real-world problems.
*   Established and monitored key performance indicators (KPIs) for AI initiatives, providing data-driven insights to evaluate success and impact, and continuously optimizing model performance for accuracy, efficiency, and scalability.
*   Implemented Machine Learning Operations (MLOps) principles, including CI/CD pipelines and automated testing, for seamless deployment and monitoring of AI models in production cloud environments, addressing performance drift and latency.
*   Communicated complex technical project solutions, requirements, and updates effectively to senior management, business stakeholders, and technical teams, translating technical concepts into clear business value.
*   Contributed to fostering innovation through data-driven approaches and leveraging cutting-edge technological advancements, ensuring alignment with organizational goals and ethical considerations.

**Education**

**Master’s degree in Computer Science**
Specialization: Big Data and Data Science
Simon Fraser University, Canada

**Technical Skills**

*   **Programming Languages:** Python, SQL, R, JavaScript
*   **Machine Learning & AI Frameworks:** TensorFlow, Generative AI (LLMs, LangChain, Retrieval-Augmented Generation - RAG, Agentic Architecture), Machine Learning Operations (MLOps), Deep Learning, Natural Language Processing (NLP)
*   **Cloud Platforms & Tools:** Experienced in deploying AI models to production cloud environments (AWS, Azure, GCP - *Implied by MLOps & production deployment*), Docker, Kubernetes (Containerization), SQL, Hadoop, Spark
*   **Data Tools & Analytics:** Tableau, Power BI, Jupyter Notebook, Large Datasets, Unstructured Data, Advanced Dashboards & Data Visualization
*   **AI Operations (AIOps):** CI/CD Pipelines, Containerization, Automated Testing, Distributed Training, Inference Optimization, Resource Utilization (GPU/TPU)
*   **System Design:** Complex System Design, Generative AI System Architecture
*   **Other Technologies:** Blockchain Technology, Information Retrieval Techniques, Knowledge Graphs, Content Management Systems

**Project Experiences & Contributions**

*   **RAG-BAML (Python):** Hands-on experience with Retrieval-Augmented Generation (RAG) using BAML, demonstrating proficiency in advanced Generative AI model development and information retrieval.
*   **RAG-Lab (Jupyter Notebook):** Further exploration and development of RAG models and experimental AI solutions.
*   **generativeAI-System-Design (Forked):** Active engagement with architectural principles and design of scalable Generative AI systems.
*   **system-design-101 (Forked):** Demonstrates commitment to mastering and elucidating complex system designs, critical for large-scale engineering endeavors and AI solutions.
*   **Agents-Lab (Jupyter Notebook):** Practical application and experimentation with AI agents, showcasing understanding of agentic architectures.
*   **Shares-Buying-Base-Chain (JavaScript):** Illustrates engagement and expertise in blockchain technology and analysis of on-chain interactions, showcasing ability to work with diverse data structures.
*   **GitHub Activity:** Maintains 73 repositories and has accumulated 42 stars, signifying active participation in the open-source community and a willingness to share work and insights.

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))